In [9]:
# pip install speechrecognition pyttsx3 googletrans==4.0.0-rc1 pyaudio

pip install pipwin
pip install pyaudio


SyntaxError: invalid syntax (1066680132.py, line 3)

In [1]:
import time
import threading
import speech_recognition as sr
from googletrans import Translator
import pyttsx3




In [ ]:
# TRANSSLATOR SUPPORT
translator = Translator()

# Text-to-speech setup
def speak(text, lang="en"):
    engine = pyttsx3.init()
    voices = engine.getProperty('voices')

    # choose English or Spanish voice if available
    if lang.startswith("es"):
        for v in voices:
            if "spanish" in v.name.lower():
                engine.setProperty('voice', v.id)
                break
    else:
        for v in voices:
            if "english" in v.name.lower():
                engine.setProperty('voice', v.id)
                break

    engine.say(text)
    engine.runAndWait()
    engine.stop()


# ====== Speech recognition =====
def transcribe_audio_from_mic(language="en-US", timeout=5, phrase_time_limit=5):
    """Record short speech and return transcribed text."""
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print(f"[ Listening for {phrase_time_limit}s in {language}]...")
        r.adjust_for_ambient_noise(source, duration=0.5)
        try:
            audio = r.listen(source, timeout=timeout, phrase_time_limit=phrase_time_limit)
        except sr.WaitTimeoutError:
            print(" No speech detected — please try speaking again.")
            return ""

    try:
        text = r.recognize_google(audio, language=language)
        print("ASR:", text)
        return text
    except sr.UnknownValueError:
        print(" Could not understand audio.")
        return ""
    except sr.RequestError as e:
        print(" API error:", e)
        return ""


# ====== Worker loops ======
def spanish_to_english_loop(stop_event):
    """Loop: listen Spanish -> translate to English -> speak English"""
    while not stop_event.is_set():
        text_sp = transcribe_audio_from_mic(language="es-ES")
        if not text_sp:
            print("[SP→EN] No speech recognized. Try again.")
            continue

        try:
            res = translator.translate(text_sp, src='es', dest='en')
            en_text = res.text
            print(f"[SP→EN] {text_sp}  ->  {en_text}")
            speak(en_text, lang='en')
        except Exception as e:
            print("Translation error:", e)


def english_to_spanish_loop(stop_event):
    """Loop: listen English -> translate to Spanish -> speak Spanish"""
    while not stop_event.is_set():
        text_en = transcribe_audio_from_mic(language="en-US")
        if not text_en:
            print("[EN→SP] No speech recognized. Try again.")
            continue

        try:
            res = translator.translate(text_en, src='en', dest='es')
            sp_text = res.text
            print(f"[EN→SP] {text_en}  ->  {sp_text}")
            speak(sp_text, lang='es')
        except Exception as e:
            print("Translation error:", e)


# ====== Main program ======
if __name__ == "__main__":
    stop_event = threading.Event()
    print(" Real-Time English ↔ Spanish Voice Translator")
    print(" Speak when prompted. Press Ctrl+C to stop.\n")

    try:
        while True:
            print("\n- Spanish speaker turn ---")
            sp = transcribe_audio_from_mic(language="es-ES")
            if sp:
                try:
                    en = translator.translate(sp, src='es', dest='en').text
                    print("[SP→EN]", sp, "->", en)
                    speak(en, lang='en')
                except Exception as e:
                    print("Translate error:", e)
            time.sleep(0.5)

            print("\n--  English speaker turn ---")
            en = transcribe_audio_from_mic(language="en-US")
            if en:
                try:
                    sp = translator.translate(en, src='en', dest='es').text
                    print("[EN→SP]", en, "->", sp)
                    speak(sp, lang='es')
                except Exception as e:
                    print("Translate error:", e)
            time.sleep(0.5)

    except KeyboardInterrupt:
        print("\n Stopping demo.")
        stop_event.set()